In [ ]:
import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)

from google.colab import drive
drive.mount('/content/drive/')
import pandas as pd
import os
os.chdir('/content/drive/MyDrive')

Mounted at /content/drive/


In [ ]:
cd CZ4034/

/content/drive/MyDrive/CZ4034


In [ ]:
df_train = pd.read_csv("./data/shared_data_train_pol.csv")
df_test = pd.read_csv("./data/shared_data_test_pol.csv")

# Train Stacked Model


In [ ]:
SVM = pd.read_csv("./results/svm_pred_train_pol.csv")
XGB = pd.read_csv("./results/xgb_pred_train_pol.csv")
NAV = pd.read_csv("./results/bayes_pred_train_pol.csv")

In [ ]:
train_X = pd.concat([SVM.iloc[:,1],NAV.iloc[:,1],XGB.iloc[:,1]],axis=1)
train_y = df_train.label

In [ ]:
SVM_test = pd.read_csv("./results/svm_pred_test_pol.csv")
XGB_test = pd.read_csv("./results/xgb_pred_test_pol.csv")
NAV_test = pd.read_csv("./results/bayes_pred_test_pol.csv")

In [ ]:
test_X = pd.concat([SVM_test.iloc[:,1],NAV_test.iloc[:,1],XGB_test.iloc[:,1]],axis=1)
test_y = df_test.label

In [ ]:
test_y = [1 if i == 1 else 0 for i in test_y]
train_y = [1 if i == 1 else 0 for i in train_y]

In [ ]:
train_X.columns = ['SVM','NAV','XGB']
test_X.columns = ['SVM','NAV','XGB']

In [ ]:
train_X = train_X.replace([2],0)
test_X = test_X.replace([2],0)

In [ ]:
train_X

,SVM,NAV,XGB
0,0,0,0
1,0,0,0
2,1,1,1
3,0,0,0
4,0,0,0
...,...,...,...
794,0,0,0
795,1,1,1
796,1,1,1
797,1,1,1


# Majority Voting


In [ ]:
test_X.mode(axis=1)

,0
0,1
1,1
2,1
3,1
4,0
...,...
183,1
184,1
185,1
186,0


In [ ]:
vote = test_X.mode(axis=1)[0]

from sklearn.metrics import confusion_matrix, classification_report
class_names = ['negative', 'positive']
print(classification_report(test_y, vote, target_names=class_names,digits=3))

              precision    recall  f1-score   support

    negative      0.918     0.947     0.932        94
    positive      0.945     0.915     0.930        94

    accuracy                          0.931       188
   macro avg      0.931     0.931     0.931       188
weighted avg      0.931     0.931     0.931       188



# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(train_X, train_y)
LR_pred = model.predict(test_X)
from sklearn.metrics import confusion_matrix, classification_report
class_names = ['negative', 'positive']
print(classification_report(test_y, LR_pred, target_names=class_names,digits = 3))

              precision    recall  f1-score   support

    negative      0.918     0.947     0.932        94
    positive      0.945     0.915     0.930        94

    accuracy                          0.931       188
   macro avg      0.931     0.931     0.931       188
weighted avg      0.931     0.931     0.931       188



# XGB

In [ ]:
!pip install xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

# A parameter grid for XGBoost
# can add more range
# params = {
#     'eta': [0.01, 0.1, 0.2],
#     'max_depth': [3, 5, 7],
#     'subsample': [0.5, 0.7, 1.0],
#     'colsample_bytree': [0.5, 0.7, 1.0],
#     'min_child_weight': [1, 5, 10],
# }

xgb = XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1)

folds = 3

# grid = GridSearchCV(estimator=xgb, param_grid=params, scoring='accuracy', cv=folds, verbose=3)
xgb.fit(train_X, train_y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
xgb_pred = xgb.predict(test_X)
from sklearn.metrics import confusion_matrix, classification_report
class_names = ['neutral', 'polarised']
print(classification_report(test_y, xgb_pred, target_names=class_names,digits=3))

              precision    recall  f1-score   support

     neutral      0.914     0.904     0.909        94
   polarised      0.905     0.915     0.910        94

    accuracy                          0.910       188
   macro avg      0.910     0.910     0.910       188
weighted avg      0.910     0.910     0.910       188



In [ ]:
xgb_pred = grid.predict(test_X)
from sklearn.metrics import confusion_matrix, classification_report
class_names = ['neutral', 'polarised']
print(classification_report(test_y, xgb_pred, target_names=class_names))

              precision    recall  f1-score   support

     neutral       0.95      0.91      0.93        94
   polarised       0.92      0.95      0.93        94

    accuracy                           0.93       188
   macro avg       0.93      0.93      0.93       188
weighted avg       0.93      0.93      0.93       188



# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# A parameter grid for XGBoost
# can add more range
params = {'bootstrap': [True, False],
 'max_depth': [10, 20, 30, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000]}

# rf_model = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
#                                               class_weight=None,
#                                               criterion='gini', max_depth=None,
#                                               max_features='auto',
#                                               max_leaf_nodes=None,
#                                               max_samples=None,
#                                               min_impurity_decrease=0.0,
#                                               min_impurity_split=None,
#                                               min_samples_leaf=1,
#                                               min_samples_split=2,
#                                               min_weight_fraction_leaf=0.0,
#                                               n_estimators=100, n_jobs=None,
#                                               random_state=None, verbose=0,
#                                               warm_start=False)

folds = 3

grid = GridSearchCV(estimator=rf_model, param_grid=params, scoring='accuracy', cv=folds, verbose=3)
rf_model.fit(train_X, train_y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
RF_pred = rf_model.predict(test_X)
from sklearn.metrics import confusion_matrix, classification_report
class_names = ['neutral', 'polarised']
print(classification_report(test_y, RF_pred, target_names=class_names,digits=3))

              precision    recall  f1-score   support

     neutral      0.914     0.904     0.909        94
   polarised      0.905     0.915     0.910        94

    accuracy                          0.910       188
   macro avg      0.910     0.910     0.910       188
weighted avg      0.910     0.910     0.910       188



In [ ]:
RF_pred = grid.predict(test_X)
from sklearn.metrics import confusion_matrix, classification_report
class_names = ['neutral', 'polarised']
print(classification_report(test_y, RF_pred, target_names=class_names))

              precision    recall  f1-score   support

     neutral       0.91      0.91      0.91        94
   polarised       0.91      0.90      0.91        94

    accuracy                           0.91       188
   macro avg       0.91      0.91      0.91       188
weighted avg       0.91      0.91      0.91       188



In [ ]:
a = train_X.values
b = (a == a[:, [0]]).all(axis=1)

In [ ]:
res = pd.DataFrame()
res['XGB'] = xgb_pred 
res['LR'] = LR_pred
res['RF'] = RF_pred


In [ ]:
a = res.values
b = (a == a[:, [0]]).all(axis=1)

In [ ]:
sum(b)

220